In [ ]:
#Install Dependencies

!pip install tensorflow

In [1]:
#Import Libraries

import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import GlobalAveragePooling2D, Dropout, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [2]:
#Mount Google Drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
DATASET_PATH = '/content/drive/MyDrive/dataset'

In [4]:
#Data Generators

IMG_SIZE = 224
BATCH_SIZE = 32

train_dir = os.path.join(DATASET_PATH, "Train")
val_dir   = os.path.join(DATASET_PATH, "Validation")
test_dir  = os.path.join(DATASET_PATH, "Test")

train_aug = ImageDataGenerator(rescale=1./255,
                               rotation_range=20,
                               zoom_range=0.15,
                               width_shift_range=0.2,
                               height_shift_range=0.2,
                               shear_range=0.15,
                               horizontal_flip=True,
                               fill_mode="nearest")

val_aug = ImageDataGenerator(rescale=1./255)

train_data = train_aug.flow_from_directory(train_dir,
                                           target_size=(IMG_SIZE, IMG_SIZE),
                                           batch_size=BATCH_SIZE,
                                           class_mode="categorical")

val_data = val_aug.flow_from_directory(val_dir,
                                       target_size=(IMG_SIZE, IMG_SIZE),
                                       batch_size=BATCH_SIZE,
                                       class_mode="categorical")

Found 1492 images belonging to 2 classes.
Found 184 images belonging to 2 classes.


In [5]:
#Build Model (2-class)

base_model = MobileNetV2(weights="imagenet", include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))
base_model.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.3)(x)
output = Dense(2, activation="softmax")(x)

model = Model(inputs=base_model.input, outputs=output)
model.compile(optimizer=Adam(1e-4), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1 (Conv2D)      │ (None, 112, 112,  │        864 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn_Conv1            │ (None, 112, 112,  │        128 │ Conv1[0][0]       │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1_relu (ReLU)   │ (None, 112, 112,  │          0 │ bn_Conv1[0][0]    │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        288 │ Conv1_relu[0][0]  │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        128 │ expanded_conv_de… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │          0 │ expanded_conv_de… │
│ (ReLU)              │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │        512 │ expanded_conv_de… │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │         64 │ expanded_conv_pr… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand      │ (None, 112, 112,  │      1,536 │ expanded_conv_pr… │
│ (Conv2D)            │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_BN   │ (None, 112, 112,  │        384 │ block_1_expand[0… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_relu │ (None, 112, 112,  │          0 │ block_1_expand_B… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_pad         │ (None, 113, 113,  │          0 │ block_1_expand_r… │
│ (ZeroPadding2D)     │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise   │ (None, 56, 56,    │        864 │ block_1_pad[0][0] │
│ (DepthwiseConv2D)   │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │        384 │ block_1_depthwis… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │          0 │ block_1_depthwis… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_project     │ (None, 56, 56,    │      2,304 │ block_1_depthwis

 Total params: 2,260,546 (8.62 MB)

 Trainable params: 2,562 (10.01 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [6]:
#Train Model

EPOCHS = 10
history = model.fit(train_data, validation_data=val_data, epochs=EPOCHS)


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
47/47 ━━━━━━━━━━━━━━━━━━━━ 566s 12s/step - accuracy: 0.6960 - loss: 0.6742 - val_accuracy: 0.8478 - val_loss: 0.3994
Epoch 2/10
47/47 ━━━━━━━━━━━━━━━━━━━━ 84s 2s/step - accuracy: 0.8097 - loss: 0.4504 - val_accuracy: 0.9076 - val_loss: 0.2974
Epoch 3/10
47/47 ━━━━━━━━━━━━━━━━━━━━ 86s 2s/step - accuracy: 0.8669 - loss: 0.3278 - val_accuracy: 0.9293 - val_loss: 0.2206
Epoch 4/10
47/47 ━━━━━━━━━━━━━━━━━━━━ 87s 2s/step - accuracy: 0.8941 - loss: 0.2812 - val_accuracy: 0.9402 - val_loss: 0.1761
Epoch 5/10
47/47 ━━━━━━━━━━━━━━━━━━━━ 87s 2s/step - accuracy: 0.9201 - loss: 0.2177 - val_accuracy: 0.9620 - val_loss: 0.1491
Epoch 6/10
47/47 ━━━━━━━━━━━━━━━━━━━━ 85s 2s/step - accuracy: 0.9483 - loss: 0.1664 - val_accuracy: 0.9674 - val_loss: 0.1278
Epoch 7/10
47/47 ━━━━━━━━━━━━━━━━━━━━ 85s 2s/step - accuracy: 0.9494 - loss: 0.1600 - val_accuracy: 0.9728 - val_loss: 0.1096
Epoch 8/10
47/47 ━━━━━━━━━━━━━━━━━━━━ 87s 2s/step - accuracy: 0.9515 - loss: 0.1451 - val_accuracy: 0.9837 - val_los

In [7]:
#Evaluate on Test Set

test_aug = ImageDataGenerator(rescale=1./255)
test_data = test_aug.flow_from_directory(test_dir,
                                         target_size=(IMG_SIZE, IMG_SIZE),
                                         batch_size=BATCH_SIZE,
                                         class_mode="categorical")

loss, acc = model.evaluate(test_data)
print(f"✅ Test Accuracy: {acc * 100:.2f}%")

Found 140 images belonging to 2 classes.
5/5 ━━━━━━━━━━━━━━━━━━━━ 8s 1s/step - accuracy: 1.0000 - loss: 0.0396
✅ Test Accuracy: 100.00%


In [8]:
#Save and Download Model

model.save("facemask_model.h5")

from google.colab import files
files.download("facemask_model.h5")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from PIL import Image
import requests
from io import BytesIO
import os

# Load trained model
model = load_model("facemask_model.h5")
class_names = ["with_mask", "without_mask"]

# Function to preprocess and predict
def predict_image(img):
    img = img.resize((224, 224))
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0) / 255.0
    pred = model.predict(img_array)[0]
    label_idx = np.argmax(pred)
    label = class_names[label_idx]
    confidence = pred[label_idx] * 100
    return label, confidence

# User input method
print("Choose input method:")
print("1. Upload an image")
print("2. Provide an image URL")

choice = input("Enter 1 or 2: ")

if choice == "1":
    from google.colab import files
    uploaded = files.upload()
    file_name = list(uploaded.keys())[0]
    image = Image.open(file_name)

elif choice == "2":
    url = input("Enter image URL: ")
    response = requests.get(url)
    image = Image.open(BytesIO(response.content))
    file_name = "url_image.jpg"
    image.save(file_name)

else:
    print("Invalid choice. Please enter 1 or 2.")
    image = None

# Predict and display
if image:
    label, confidence = predict_image(image)

    plt.imshow(image)
    plt.title(f"Prediction: {label} ({confidence:.2f}%)")
    plt.axis(False)
    plt.show()

    print(f"\n✅ Prediction: {label}")
    print(f"🎯 Confidence: {confidence:.2f}%")
